In [49]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt

In [50]:
pb.set_logging_level("INFO")

#Define the parameter values chemistry
parameter_values = pb.ParameterValues("Chen2020")

#Define the model
spm = pybamm.lithium_ion.SPM()

# Experiment

experiment = pybamm.Experiment(["Discharge at 1C until 2.5 V"])


2022-11-30 12:21:21.390 - [INFO] base_model._build_model(572): Start building Single Particle Model
2022-11-30 12:21:21.436 - [INFO] base_battery_model.build_model(1019): Finish building Single Particle Model


In [51]:
spm.variables.search("concentration")

Average negative particle concentration
Average negative particle concentration [mol.m-3]
Average positive particle concentration
Average positive particle concentration [mol.m-3]
Dead lithium concentration
Dead lithium concentration [mol.m-3]
Electrolyte concentration
Electrolyte concentration [Molar]
Electrolyte concentration [mol.m-3]
Inner SEI concentration [mol.m-3]
Inner SEI on cracks concentration [mol.m-3]
Lithium plating concentration
Lithium plating concentration [mol.m-3]
Maximum negative particle concentration
Maximum negative particle concentration [mol.m-3]
Maximum negative particle surface concentration
Maximum negative particle surface concentration [mol.m-3]
Maximum positive particle concentration
Maximum positive particle concentration [mol.m-3]
Maximum positive particle surface concentration
Maximum positive particle surface concentration [mol.m-3]
Minimum negative particle concentration
Minimum negative particle concentration [mol.m-3]
Minimum negative particle surf

In [52]:
cpe = spm.variables["Positive electrode volume-averaged concentration [mol.m-3]"]
cxa = spm.variables["X-averaged positive particle concentration [mol.m-3]"]


In [53]:
cpe.domain

['current collector']

In [54]:
cxa.domain

['positive particle']

In [55]:
for var, npts in spm.default_var_pts.items():
    print(var,'has', npts,'mesh points')

x_n has 20 mesh points
x_s has 20 mesh points
x_p has 20 mesh points
r_n has 20 mesh points
r_p has 20 mesh points
r_n_prim has 20 mesh points
r_p_prim has 20 mesh points
r_n_sec has 20 mesh points
r_p_sec has 20 mesh points
y has 10 mesh points
z has 10 mesh points
R_n has 30 mesh points
R_p has 30 mesh points


In [56]:
sim = pybamm.Simulation(spm)
sim.solve(t_eval=[0,3600])

2022-11-30 12:26:56.943 - [INFO] parameter_values.process_model(371): Start setting parameters for Single Particle Model
2022-11-30 12:26:57.015 - [INFO] parameter_values.process_model(474): Finish setting parameters for Single Particle Model
2022-11-30 12:26:57.018 - [INFO] discretisation.process_model(149): Start discretising Single Particle Model
2022-11-30 12:26:57.021 - [INFO] discretisation.remove_independent_variables_from_rhs(1199): removing variable Discharge capacity [A.h] from rhs
2022-11-30 12:26:57.267 - [INFO] discretisation.process_model(271): Finish discretising Single Particle Model
2022-11-30 12:26:57.268 - [INFO] base_solver.solve(703): Start solving Single Particle Model with CasADi solver with 'safe' mode
2022-11-30 12:26:57.271 - [INFO] base_solver.set_up(110): Start solver set-up
2022-11-30 12:26:57.294 - [INFO] base_solver.set_up(235): Finish solver set-up
2022-11-30 12:26:57.315 - [INFO] base_solver.solve(936): Finish solving Single Particle Model (the solver s

In [57]:
#Plot variables depending on R
output_vars = [
    
               "X-averaged positive particle concentration [mol.m-3]",
               "X-averaged negative particle concentration [mol.m-3]",
               "Terminal voltage [V]",
               "Positive particle surface concentration [mol.m-3]",
               "Negative particle surface concentration [mol.m-3]",
               "Electrolyte concentration [mol.m-3]",
               
              ]

# plot
sim.plot(output_variables = output_vars)

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

In [ ]:
# plot
pb.dynamic_plot(
    sols,
    [
        "Terminal voltage [V]",
        "Electrolyte potential [V]",
    ],
)

In [26]:
pb.dynamic_plot(sols, [
    "Positive particle surface concentration [mol.m-3]",
    "Negative particle surface concentration [mol.m-3]",
    "Electrolyte concentration [mol.m-3]",
    ]
 )

interactive(children=(FloatSlider(value=0.0, description='t', max=3567.8533292781535, step=35.67853329278154),…

In [59]:
pb.Solution(
     [1,45], 
     [[2, 1],[2, 1]], 
     models, 
     ["Positive particle surface concentration [mol.m-3]"], 
     t_event=None, 
     y_event=None, 
     termination='final time', 
     sensitivities=False, 
     check_solution=True
 )

TypeError: list indices must be integers or slices, not tuple